# Train a SCCNN

In this notebook, we will create and train a convolutional neural network in the simplicial complex domain, as proposed in the paper by [Yang et. al : Convolutional Learning on Simplicial Complexes (2023)](https://arxiv.org/abs/2301.11163). 

We train the model to perform complex classification using the shrec16 benchmark dataset. 

## Simplicial Complex Convolutional Neural Networks <a href="https://arxiv.org/pdf/2301.11163.pdf">[SCCNN]</a>

SCCNN extends the SCNN to the complex domain by accounting for inter-simplicial connections, i.e., contributions from simplices of adjacent orders. 

For example, we consider SCCNN layers in an SC of order two. At layer $t$, given the inputs on nodes, edges and faces, $\mathbf{h}_{t}^0,\mathbf{h}_{t}^1$ and $\mathbf{h}_{t}^2$, the SCCNN layer contains the following
$$
    \mathbf{h}_{t+1}^1 = \sigma \bigg[ \mathbf{F}_{t,\downarrow} \mathbf{B}_{1}^\top \mathbf{h}_{t}^{0} + \mathbf{F}_{t} \mathbf{h}_t^1 + \mathbf{F}_{t,\uparrow} \mathbf{B}_{2}  \mathbf{h}_t^{2} \bigg] 
$$
where $\mathbf{F}_t$ is the simplicial convolutional filter defined in the edge space, and $\mathbf{F}_{t,\downarrow}$ and $\mathbf{F}_{t,\uparrow}$ are the convolutional filters based on, respectively, only the lower and upper Laplacians. They are given by 
$$
    \mathbf{F}_{t} = {\theta}_t + \sum_{p_d=1}^{P_d} {\theta}_{t,p_d} (\mathbf{L}_{\downarrow,1})^{p_d}  + \sum_{p_u=1}^{P_u} {\theta}_{t,p_u}  (\mathbf{L}_{\uparrow,1})^{p_u} 
$$
$$
    \mathbf{F}_{t,\downarrow} = {\theta}_t + \sum_{p_d=1}^{P_d} {\theta}_{t,p_d} (\mathbf{L}_{\downarrow,1})^{p_d}  
    \text{ and } 
    \mathbf{F}_{t,\uparrow} = {\theta}_t + \sum_{p_u=1}^{P_u} {\theta}_{t,p_u}  (\mathbf{L}_{\uparrow,1})^{p_u} 
$$

Likewise, for the node output, we have 
$$
    \mathbf{h}_{t+1}^0 = \sigma \bigg[ \mathbf{F}_{t} \mathbf{h}_t^0 + \mathbf{F}_{t,\uparrow} \mathbf{B}_{1}  \mathbf{h}_t^{1} \bigg]
$$
where $\mathbf{F}_t$ and $\mathbf{F}_{t,\uparrow}$ are two graph filters essentially. 

For the face output, we have 
$$
    \mathbf{h}_{t+1}^2 = \sigma \bigg[ \mathbf{F}_{t} \mathbf{h}_{t}^{2}  + \mathbf{F}_{t,\downarrow} \mathbf{B}_{2}^\top  \mathbf{h}_t^{1} \bigg]
$$
where $\mathbf{F}_t$ and $\mathbf{F}_{t,\downarrow}$ are two simplicial filters defined in the triangle (face) space. 

In [53]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from toponetx import SimplicialComplex
import toponetx.datasets as datasets
from topomodelx.nn.simplicial.sccnn_layer import SCCNNLayer

## Pre-processing

### Import shrec dataset ##

We must first lift our graph dataset into the simplicial complex domain.

In [54]:
shrec, _ = datasets.mesh.shrec_16(size="small")
shrec = {key: np.array(value) for key, value in shrec.items()}
x_0s = shrec["node_feat"]
x_1s = shrec["edge_feat"]
x_2s = shrec["face_feat"]

ys = shrec["label"]
simplexes = shrec["complexes"]

Loading shrec 16 small dataset...

done!


In [55]:
in_channels_0 = x_0s[-1].shape[1]
in_channels_1 = x_1s[-1].shape[1]
in_channels_2 = x_2s[-1].shape[1]

in_channels_all = (in_channels_0,in_channels_1,in_channels_2)
print(in_channels_all)

(6, 10, 7)


### Define Neighborhood Strctures
Get incidence matrices $\mathbf{B}_1,\mathbf{B}_2$ and Hodge Laplacians $\mathbf{L}_0, \mathbf{L}_1$ and $\mathbf{L}_2$.

Note that the original paper considered the weighted versions of these operators. However, the current TOPONETX package does not provide such feature yet.

In [56]:
max_rank = 2 # the order of the SC is two 
incidence_1_list = []
incidence_2_list = []

laplacian_0_list = []
laplacian_down_1_list = []
laplacian_up_1_list = []
laplacian_2_list = []
 
for simplex in simplexes: 
    incidence_1 = simplex.incidence_matrix(rank=1)
    incidence_2 = simplex.incidence_matrix(rank=2)
    laplacian_0 = simplex.hodge_laplacian_matrix(rank=0,weight=True)
    laplacian_down_1 = simplex.down_laplacian_matrix(rank=1,weight=True)
    laplacian_up_1 = simplex.up_laplacian_matrix(rank=1,weight=True)
    laplacian_2 = simplex.hodge_laplacian_matrix(rank=2,weight=True)
    
    incidence_1 = torch.from_numpy(incidence_1.todense()).to_sparse()
    incidence_2 = torch.from_numpy(incidence_2.todense()).to_sparse()
    laplacian_0 = torch.from_numpy(laplacian_0.todense()).to_sparse()
    laplacian_down_1 = torch.from_numpy(laplacian_down_1.todense()).to_sparse()
    laplacian_up_1 = torch.from_numpy(laplacian_up_1.todense()).to_sparse()
    laplacian_2 = torch.from_numpy(laplacian_2.todense()).to_sparse()
    
    incidence_1_list.append(incidence_1)
    incidence_2_list.append(incidence_2)
    laplacian_0_list.append(laplacian_0)
    laplacian_down_1_list.append(laplacian_down_1)
    laplacian_up_1_list.append(laplacian_up_1)
    laplacian_2_list.append(laplacian_2)

# Create the SCCNN

In [57]:
class SCCNN(torch.nn.Module):
    """SCCNN implementation for complex classification  
    Note: In this task, we can consider the output on any order of simplices for the classification task, which of course can be amended by a readout layer.                                        

    Parameters
    ----------
    in_channels_all: tuple of int
        Dimension of input features on (nodes, edges, faces)
    intermediate_channels_all: tuple of int
        Dimension of features of intermediate layers on (nodes, edges, faces)
    out_channels_all: tuple of int
        Dimension of output features on (nodes, edges, faces)
    conv_order: int
        Order of convolutions, we consider the same order for all convolutions 
    sc_order: int
        SC order 
    n_layers: int
        Numer of layers 

    """
    def __init__(self, 
                 in_channels_all, 
                 intermediate_channels_all, 
                 out_channels_all, 
                 conv_order, sc_order, 
                 aggr_norm=False, update_func=None, 
                 num_classes = 1,
                 n_layers=2):
        super().__init__()
        # first layer
        # we use an MLP to map the features on simplices of different dimensions to the same dimension   
        self.in_linear_0 = torch.nn.Linear(in_channels_all[0],intermediate_channels_all[0])
        self.in_linear_1 = torch.nn.Linear(in_channels_all[1],intermediate_channels_all[1])
        self.in_linear_2 = torch.nn.Linear(in_channels_all[2],intermediate_channels_all[2])

        layers = []
        for _ in range(n_layers):
            layers.append(
                SCCNNLayer(in_channels=intermediate_channels_all,out_channels=out_channels_all,conv_order=conv_order,sc_order=sc_order,aggr_norm=aggr_norm,update_func=update_func)
            )
    
        self.layers = torch.nn.ModuleList(layers)  

        out_channels_0, out_channels_1, out_channels_2 = out_channels_all
        self.out_linear_0 = torch.nn.Linear(out_channels_0,num_classes)
        self.out_linear_1 = torch.nn.Linear(out_channels_1,num_classes)
        self.out_linear_2 = torch.nn.Linear(out_channels_2,num_classes)
        

    def forward(self,x_all,laplacian_all,incidence_all):
        """Forward computation. 
        
        Parameters
        ---------
        x: tuple tensors
            (node, edge, face) features 
            each entry shape = [n_simplices, channels]
      
        laplacian: tuple of tensors
            (graph laplacian L0, down edge laplacian L1_d, upper edge laplacian L1_u, face laplacian L2)
            each entry shape = [n_simplices,n_simplices]

        incidence_1: tuple of tensors
            tuple pf order 1 and 2 incidence matrices 
            shape of B1 = [n_nodes, n_edges]
            shape of B2 = [n_edges, n_faces]
    
        Returns        
        -------
        _ : tensor, shape = [1]
            Label assigned to whole complex.
        """    
        x_0, x_1, x_2 = x_all
        in_x_0 = self.in_linear_0(x_0)
        in_x_1 = self.in_linear_1(x_1)
        in_x_2 = self.in_linear_2(x_2)

        # Forward through SCCNN 
        x_all = (in_x_0,in_x_1,in_x_2)
        for layer in self.layers:
            x_all = layer(x_all,laplacian_all,incidence_all)
        
        """
        We pass the output on the nodes, edges and triangles to a linear layer and use the sum of the averages of outputs on each simplex for labels of complex
        """
        x_0, x_1, x_2 = x_all 

        x_0 = self.out_linear_0(x_0)
        x_1 = self.out_linear_1(x_1)
        x_2 = self.out_linear_2(x_2)
 
        # Take the average of the 2D, 1D, and 0D cell features. If they are NaN, convert them to 0.
        two_dimensional_cells_mean = torch.nanmean(x_2, dim=0)
        two_dimensional_cells_mean[torch.isnan(two_dimensional_cells_mean)] = 0
        one_dimensional_cells_mean = torch.nanmean(x_1, dim=0)
        one_dimensional_cells_mean[torch.isnan(one_dimensional_cells_mean)] = 0
        zero_dimensional_cells_mean = torch.nanmean(x_0, dim=0)
        zero_dimensional_cells_mean[torch.isnan(zero_dimensional_cells_mean)] = 0
        # Return the sum of the averages
        return  (
            two_dimensional_cells_mean
            + one_dimensional_cells_mean
            + zero_dimensional_cells_mean
        )

# Train the Neural Network

We specify the model with our pre-made neighborhood structures and specify an optimizer.

In [61]:
conv_order = 2
intermediate_channels_all = (4,4,4)
out_channels_all = intermediate_channels_all
num_layers = 2

model = SCCNN(in_channels_all=in_channels_all,intermediate_channels_all=intermediate_channels_all,out_channels_all=out_channels_all,conv_order=conv_order,sc_order=max_rank,num_classes=1,n_layers=num_layers)

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
loss_fn = torch.nn.MSELoss(size_average=True,reduction='mean')
print(model)

SCCNN(
  (in_linear_0): Linear(in_features=6, out_features=4, bias=True)
  (in_linear_1): Linear(in_features=10, out_features=4, bias=True)
  (in_linear_2): Linear(in_features=7, out_features=4, bias=True)
  (layers): ModuleList(
    (0-1): 2 x SCCNNLayer()
  )
  (out_linear_0): Linear(in_features=4, out_features=1, bias=True)
  (out_linear_1): Linear(in_features=4, out_features=1, bias=True)
  (out_linear_2): Linear(in_features=4, out_features=1, bias=True)
)


/home/nfs/mmaosheng/.local/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [62]:
test_size = 0.2
x_0_train, x_0_test = train_test_split(x_0s, test_size=test_size, shuffle=False)
x_1_train, x_1_test = train_test_split(x_1s, test_size=test_size, shuffle=False)
x_2_train, x_2_test = train_test_split(x_2s, test_size=test_size, shuffle=False)

incidence_1_train, incidence_1_test = train_test_split(
    incidence_1_list, test_size=test_size, shuffle=False
)
incidence_2_train, incidence_2_test = train_test_split(incidence_2_list, test_size=test_size, shuffle=False)
laplacian_0_train, laplacian_0_test = train_test_split(laplacian_0_list, test_size=test_size, shuffle=False)
laplacian_down_1_train, laplacian_down_1_test = train_test_split(laplacian_down_1_list, test_size=test_size, shuffle=False)
laplacian_up_1_train, laplacian_up_1_test = train_test_split(laplacian_up_1_list, test_size=test_size, shuffle=False)
laplacian_2_train, laplacian_2_test = train_test_split(laplacian_2_list, test_size=test_size, shuffle=False)

y_train, y_test = train_test_split(ys, test_size=test_size, shuffle=False)

We train the SCCNN using low amount of epochs: we keep training minimal for the purpose of rapid testing.

In [63]:
test_interval = 1
num_epochs = 5

for epoch_i in range(1, num_epochs + 1):
    epoch_loss = []
    model.train()
    for x_0, x_1, x_2, incidence_1, incidence_2, laplacian_0, laplacian_down_1, laplacian_up_1, laplacian_2, y in zip(x_0_train, x_1_train, x_2_train, incidence_1_train, incidence_2_train, laplacian_0_train, laplacian_down_1_train, laplacian_up_1_train, laplacian_2_train, y_train):
        
        x_0 = torch.tensor(x_0)
        x_1 = torch.tensor(x_1)
        x_2 = torch.tensor(x_2)
        y = torch.tensor(y,dtype=torch.float)
        optimizer.zero_grad()
        x_all = (x_0.float(),x_1.float(),x_2.float())
        laplacian_all = (laplacian_0, laplacian_down_1, laplacian_up_1, laplacian_2)
        incidence_all = (incidence_1, incidence_2)

        y_hat = model(x_all, laplacian_all, incidence_all)

        # print(y_hat) 
        loss = loss_fn(y_hat, y)

        epoch_loss.append(loss.item())
        loss.backward()
        optimizer.step()    

    print(
        f"Epoch: {epoch_i} loss: {np.mean(epoch_loss):.4f}",
        flush=True,
    )
    if epoch_i % test_interval == 0:
        with torch.no_grad():
            for x_0, x_1, x_2, incidence_1, incidence_2, laplacian_0, laplacian_down_1, laplacian_up_1, laplacian_2, y in zip(x_0_test, x_1_test, x_2_test, incidence_1_test, incidence_2_test, laplacian_0_test, laplacian_down_1_test, laplacian_up_1_test, laplacian_2_test, y_test):
    
                x_0 = torch.tensor(x_0)
                x_1 = torch.tensor(x_1)
                x_2 = torch.tensor(x_2)
                y = torch.tensor(y,dtype=torch.float)
                optimizer.zero_grad()
                x_all = (x_0.float(),x_1.float(),x_2.float())
                laplacian_all = (laplacian_0, laplacian_down_1, laplacian_up_1, laplacian_2)
                incidence_all = (incidence_1, incidence_2)

                y_hat = model(x_all, laplacian_all, incidence_all)
                    
                loss = loss_fn(y_hat, y)
            print(f"Test_loss: {loss:.4f}", flush=True)

/home/nfs/mmaosheng/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/nfs/mmaosheng/.local/lib/python3.10/site-packages/torch/optim/optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():


Epoch: 1 loss: 624748.2849
Test_loss: 831.5817
Epoch: 2 loss: 131.9210
Test_loss: 127.5255
Epoch: 3 loss: 97.9960
Test_loss: 116.0045
Epoch: 4 loss: 90.4851
Test_loss: 78.6304
Epoch: 5 loss: 87.1210
Test_loss: 50.7896
